# SOPHIE vs traditional DE

The goal of this experiment is to determine how often SOPHIE ranks specific genes over generic genes compared to using traditional DE analysis

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import numpy as np
import pandas as pd
import random
from ponyo import utils

np.random.seed(1)

## Create simulated data

1. Each perturbation experiment has 4 samples (2 perturbed vs 2 control) with 1000 genes. Create initial expression profiles for 4 samples by drawing from a gaussian with a mean/sd for each gene.
2. Say that there are 100 generic genes. The generic genes will have the same scalar value added for the “perturbed” samples
3. Each perturbation experiment will have 10 “specific” genes perturbed in addition to the “generic” ones. Select specific genes randomly . Then the specific genes will have the same scalar value added for the “perturbed” samples
4. Repeat this process 90 times to get 90 experiments

In [2]:
# User params
num_genes = 1000
num_samples = 4
num_generic_genes = 100
num_specific_genes = 10
num_experiments = 90

# params for distribution for scaler used to shift generic and specific genes
mean_scaler = 0
var_scaler = 10

In [3]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = "config_sophie_vs_trad.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Local directory to store intermediate files
local_dir = params["local_dir"]

# Un-normalized compendium filename
raw_compendium_filename = params["raw_compendium_filename"]

In [5]:
# Pickle files to save
generic_gene_ids_filename = "generic_gene_ids.pickle"

In [6]:
# Setup variables

# Sample ids (2 controls, 2 perturbed)
sample_ids = ["0_control", "1_control", "2_perturb", "3_perturb"]

# Make range of numbers into string to use as gene ids
gene_ids = [f"G_{i}" for i in range(num_genes)]

# Randomly select generic gene ids
generic_gene_ids = random.sample(gene_ids, num_generic_genes)
print(generic_gene_ids)

['G_668', 'G_229', 'G_341', 'G_15', 'G_314', 'G_925', 'G_57', 'G_198', 'G_87', 'G_47', 'G_567', 'G_86', 'G_75', 'G_246', 'G_974', 'G_659', 'G_322', 'G_495', 'G_203', 'G_458', 'G_0', 'G_192', 'G_954', 'G_296', 'G_490', 'G_62', 'G_843', 'G_480', 'G_810', 'G_872', 'G_670', 'G_39', 'G_880', 'G_732', 'G_336', 'G_990', 'G_335', 'G_181', 'G_389', 'G_904', 'G_418', 'G_390', 'G_379', 'G_225', 'G_936', 'G_373', 'G_746', 'G_724', 'G_352', 'G_313', 'G_95', 'G_551', 'G_583', 'G_71', 'G_890', 'G_778', 'G_287', 'G_40', 'G_63', 'G_577', 'G_179', 'G_637', 'G_850', 'G_718', 'G_830', 'G_164', 'G_912', 'G_629', 'G_725', 'G_110', 'G_200', 'G_861', 'G_918', 'G_806', 'G_541', 'G_646', 'G_466', 'G_811', 'G_213', 'G_821', 'G_238', 'G_621', 'G_76', 'G_180', 'G_570', 'G_261', 'G_558', 'G_574', 'G_698', 'G_662', 'G_712', 'G_794', 'G_818', 'G_516', 'G_329', 'G_728', 'G_210', 'G_840', 'G_223', 'G_823']


## Supporting functions

In [7]:
# Function to make an individual experiment
def run_make_experiment(
    num_samples,
    num_genes,
    sample_ids,
    all_gene_ids,
    generic_gene_ids,
    specific_gene_ids,
    generic_scaler,
    specific_scaler,
):
    experiment_data = {}
    for sample in range(num_samples):

        # Randomly select a different probability for each sample
        p = random.uniform(0.0, 1.0)
        sample_profile = np.random.negative_binomial(1000, p, num_genes)

        # Create dictionary to define dataframe
        experiment_data[sample_ids[sample]] = sample_profile

    # Create experiment dataframe
    experiment_df = pd.DataFrame(experiment_data).T

    # Set column header
    experiment_df.columns = all_gene_ids

    # Perturb generic genes by scaler
    # Randomly select a scaler
    # Only add scaler to perturbed samples
    experiment_df.loc[
        experiment_df.index.str.contains("perturb"), generic_gene_ids
    ] += generic_scaler

    # Perturb specific genes by a different scaler
    experiment_df.loc[
        experiment_df.index.str.contains("perturb"), specific_gene_ids
    ] += specific_scaler

    return experiment_df

In [8]:
# Make multiple experiments
def make_experiments(
    num_experiments,
    num_samples,
    num_genes,
    sample_ids,
    all_gene_ids,
    generic_gene_ids,
    num_specific_genes,
):

    expression_df = pd.DataFrame()
    specific_gene_id_lst = []

    for i in range(num_experiments):

        # Randomly select specific genes from the pool of (900)remaining genes
        # Select without replacement
        remaining_gene_ids = list(set(all_gene_ids).difference(generic_gene_ids))
        specific_gene_ids = random.sample(remaining_gene_ids, num_specific_genes)

        # Save specific gene ids for reference later
        specific_gene_id_lst.append(specific_gene_ids)

        # Randomly select scaler for generic and specific genes from the same distribution
        scale_factors = np.random.randint(mean_scaler, var_scaler, size=2)
        generic_scaler = scale_factors[0]
        specific_scaler = scale_factors[1]

        experiment_df = run_make_experiment(
            num_samples,
            num_genes,
            sample_ids,
            all_gene_ids,
            generic_gene_ids,
            specific_gene_ids,
            generic_scaler,
            specific_scaler,
        )

        # Concatenate experiments
        expression_df = pd.concat([expression_df, experiment_df])

    # Try to reset index to see if this makes a difference
    # NOTE: VAE don't train when sample indices are identical, not sure why
    if num_experiments > 1:
        expression_df = expression_df.reset_index(drop=True)

    return expression_df, specific_gene_id_lst

## Make a template experiment

In [9]:
for i in range(10):
    template_experiment, template_specific_gene_ids = make_experiments(
        1,
        num_samples,
        num_genes,
        sample_ids,
        gene_ids,
        generic_gene_ids,
        num_specific_genes,
    )

    # Save template experiment
    raw_template_filename = f"/home/alexandra/Documents/Data/Generic_expression_patterns/reviewer_experiment/raw_template_{i}.tsv"
    template_experiment.to_csv(raw_template_filename, sep="\t")

    # Pickle specific gene ids
    template_specific_gene_ids_filename = f"/home/alexandra/Documents/Data/Generic_expression_patterns/reviewer_experiment/template_specific_gene_ids_{i}.pickle"
    with open(template_specific_gene_ids_filename, "wb") as pkl_fh:
        pickle.dump(template_specific_gene_ids[0], pkl_fh, protocol=3)

In [10]:
print(template_experiment.shape)
template_experiment.head()

(4, 1000)


,G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,...,G_990,G_991,G_992,G_993,G_994,G_995,G_996,G_997,G_998,G_999
0_control,513,487,501,513,414,499,479,460,441,458,...,469,514,485,466,453,474,490,487,505,486
1_control,8024,8033,8367,8303,8312,8320,8405,8217,8130,8221,...,8528,8879,8307,9036,8180,8867,8158,8371,8233,8323
2_perturb,4714,4728,5067,4889,4933,4995,4786,5134,4971,5024,...,4954,4612,4999,5075,5086,4706,5213,4730,5151,5105
3_perturb,2778,2805,2814,2842,2895,2781,2931,2951,2901,2872,...,2900,2792,2934,2793,2934,2741,2724,2788,2611,2914


In [11]:
template_experiment[generic_gene_ids]

,G_668,G_229,G_341,G_15,G_314,G_925,G_57,G_198,G_87,G_47,...,G_712,G_794,G_818,G_516,G_329,G_728,G_210,G_840,G_223,G_823
0_control,484,493,473,511,488,508,438,461,478,487,...,469,485,490,486,517,488,472,486,469,507
1_control,8549,8826,8089,8063,8125,8422,8447,8296,7988,8293,...,8572,8029,8554,8664,8002,7641,8439,8278,8866,8123
2_perturb,5153,4695,4747,4872,4867,4699,4887,4779,4650,4899,...,4922,5031,4636,4979,4953,4886,4640,5100,4907,4966
3_perturb,2911,2962,2759,2872,2980,2836,3113,2973,2834,2834,...,2734,2795,2751,2826,2918,2895,2971,2877,2894,2734


In [12]:
template_specific_gene_ids[0]

['G_703',
 'G_775',
 'G_312',
 'G_590',
 'G_554',
 'G_636',
 'G_862',
 'G_795',
 'G_154',
 'G_598']

## Make compendium

In [13]:
compendium, compendium_specific_ids = make_experiments(
    num_experiments,
    num_samples,
    num_genes,
    sample_ids,
    gene_ids,
    generic_gene_ids,
    num_specific_genes,
)

In [14]:
print(compendium.shape)
compendium.head()

(360, 1000)


,G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,...,G_990,G_991,G_992,G_993,G_994,G_995,G_996,G_997,G_998,G_999
0,710,712,653,724,691,757,672,744,691,654,...,730,737,699,712,703,756,696,696,691,761
1,2126,2163,2151,2206,2095,2094,2113,2168,2178,2078,...,2096,2123,2163,2246,2010,2200,1884,2072,2032,2134
2,2728,2563,2632,2608,2518,2491,2585,2722,2429,2546,...,2514,2636,2565,2567,2470,2422,2601,2580,2419,2552
3,527,569,507,505,523,495,610,535,615,543,...,495,533,521,566,524,587,576,542,569,563
4,740,765,726,734,677,716,759,773,733,815,...,752,776,750,774,733,742,735,767,740,773


In [15]:
compendium_specific_ids

[['G_648',
  'G_462',
  'G_803',
  'G_735',
  'G_393',
  'G_237',
  'G_819',
  'G_8',
  'G_552',
  'G_770'],
 ['G_508',
  'G_610',
  'G_298',
  'G_549',
  'G_77',
  'G_768',
  'G_877',
  'G_339',
  'G_679',
  'G_41'],
 ['G_404',
  'G_571',
  'G_90',
  'G_131',
  'G_52',
  'G_626',
  'G_560',
  'G_438',
  'G_274',
  'G_54'],
 ['G_370',
  'G_115',
  'G_242',
  'G_159',
  'G_382',
  'G_957',
  'G_727',
  'G_122',
  'G_582',
  'G_158'],
 ['G_30',
  'G_272',
  'G_100',
  'G_413',
  'G_800',
  'G_170',
  'G_475',
  'G_26',
  'G_137',
  'G_960'],
 ['G_561',
  'G_575',
  'G_178',
  'G_346',
  'G_132',
  'G_250',
  'G_803',
  'G_370',
  'G_442',
  'G_877'],
 ['G_176',
  'G_707',
  'G_358',
  'G_550',
  'G_674',
  'G_599',
  'G_762',
  'G_312',
  'G_266',
  'G_118'],
 ['G_260',
  'G_710',
  'G_971',
  'G_563',
  'G_124',
  'G_658',
  'G_776',
  'G_986',
  'G_346',
  'G_153'],
 ['G_80',
  'G_693',
  'G_963',
  'G_972',
  'G_632',
  'G_756',
  'G_550',
  'G_501',
  'G_331',
  'G_652'],
 ['G_620',


In [16]:
# Save
compendium.to_csv(raw_compendium_filename, sep="\t")

# Save generic genes
# Pickle `scaler` as `scaler_filename` on disk
with open(generic_gene_ids_filename, "wb") as pkl_fh:
    pickle.dump(generic_gene_ids, pkl_fh, protocol=3)